## Схема проекта

**Принципиальные моменты:**
*   У страховой компании нет времени "на подумать", решение о приеме на страхование или отказе, а также о тарифе должено быть сделано непосредственно в момент обращения клиента за полисом. Соответственно, сервис должен работать в режиме реального времени и давать ответ в течение максимум нескольких секунд.
*   Сервис скоринга строится обычно отдельным блоком, а фронт-система взаимодействует с сервисом через API (Application programming interface).
*   Запрос данных из внешних источников может производиться как на уровне фронт-системы, так и на уровне сервиса.
*   Один из возможных вариантов - Flask.
*   Передача информации производится путем обмена json-файлами.
*   Для тестирования можно использовать Postman.

  ![](https://drive.google.com/uc?export=view&id=1OAOF1M2U14UJWDmeJg2mwo-pgSwyNyzc)

## JSON

JSON (JavaScript Object Notation) - простой текстовый формат обмена данными, он основан на подмножестве языка программирования JavaScript.<br/>
Например, строка из нашего датасета выглядела бы следующим образом:<br/>
```
{
"ID": 1,
"Exposure": 0.583,
"RecordBeg": "2004-06-01",
"RecordEnd": "",
"DrivAge": 55,
"Gender": "Female",
...
}
```

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование, запись котировок.

**Особенности:**
*   Библиотека H2O использует виртуальную Java-машину:
    *   ее нужно инициализировать один раз, а не поднимать заново для каждого расчета;
    *   вручную выделить под нее отдельный порт и указать размер используемой памяти;
    *   не создавать каждый раз заново H2O-Frame, а записывать в единожды подготовленный.
*   Необходимо удостовериться, что на всех этапах сервис отрабатывает корректно; например, можно иметь заготовленный массив котировок с заранее известными ответами.
*   Необходимо провести нагрузочное тестирование и удостовериться, что сервис справляется с нагрузкой.
*   Для согласованности версий Python, Java при переносе на другие серверы имеет смысл использовать докеры.



## Flask

In [1]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Google Colab provides a virtual machine so we cannot access the localhost as we do on our local machine when running a local web server. What we can do is expose it to a public URL using ngrok.
https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [2]:
!pip install flask-ngrok

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [10]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e7b6460f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Dec/2019 17:37:02] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2019 17:37:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2019 17:41:40] "GET /a HTTP/1.1" 200 -


In [4]:
!pip install h2o

     |████████████████████████████████| 123.4MB 91kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.1-py2.py3-none-any.whl size=123410554 sha256=0bceb68d14d2dafabac482444c80d1dc61f66a840f6ac73f17f59433bbcbe8f8
  Stored in directory: /root/.cache/pip/wheels/ff/82/f4/3fe9c895b93c53b25ae44dc44aa54dad571c711f48bc485fe9
Successfully built h2o


In [5]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.5" 2019-10-15; OpenJDK Runtime Environment (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04); OpenJDK 64-Bit Server VM (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpkvjsfjz_
  JVM stdout: /tmp/tmpkvjsfjz_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpkvjsfjz_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_unknownUser_3xtb54
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [11]:
# Загружаем обученные модели

model_glm_poisson = h2o.load_model('/content/drive/My Drive/Colab Notebooks/GLM_model_python_1573818197972_1')
model_glm_gamma = h2o.load_model('/content/drive/My Drive/Colab Notebooks/GLM_model_python_1573818197972_2')

H2OResponseError: ignored

In [0]:
# Загружаем обученные модели

model_glm_poisson = h2o.load_model('/content/drive/My Drive/Colab Notebooks/GLM_model_python_1577365406307_1')
model_glm_gamma = h2o.load_model('/content/drive/My Drive/Colab Notebooks/GLM_model_python_1577365406307_2')

In [13]:
# Обработчики и запуск Flask

def map_for_dict_Gender(Gender):
  dict_Gender = {'Male':0, 'Female':1}
  res = dict_Gender.get(Gender)
  return res

def map_for_dict_MariStat(MariStat):
  dict_MariStat = {'Other':0, 'Alone':1}
  res = dict_MariStat.get(MariStat)
  return res

def f_VehUsage_Professional(VehUsage):
  if VehUsage == 'Professional':
    VehUsage_Professional = 1
  else:
    VehUsage_Professional = 0
  return(VehUsage_Professional)

def f_VehUsage_Private_trip_to_office(VehUsage):
  if VehUsage == 'Private+trip to office':
    VehUsage_Private_trip_to_office = 1
  else:
    VehUsage_Private_trip_to_office = 0
  return(VehUsage_Private_trip_to_office)

def f_VehUsage_Private(VehUsage):
  if VehUsage == 'Private':
    VehUsage_Private = 1
  else:
    VehUsage_Privatel = 0
  return(VehUsage_Private)

def f_VehUsage_Professional_run(VehUsage):
  if VehUsage == 'Professional run':
    VehUsage_Professional_run = 1
  else:
    VehUsage_Professional_run = 0
  return(VehUsage_Professional_run)

def return_NewH2o_Frame():
  columns = [
      'LicAge',
      'Gender',
      'MariStat',
      'DrivAge',
      'HasKmLimit',
      'BonusMalus',
      'OutUseNb',
      'RiskArea',
      'VehUsg_Private',
      'VehUsg_Private+trip to office',
      'VehUsg_Professional',
      'VehUsg_Professional run',
      'CSP1',
      'CSP2',
      'CSP3',
      'CSP6',
      'CSP7',
      'CSP20',
      'CSP21',
      'CSP22',
      'CSP26',
      'CSP37',
      'CSP40',
      'CSP42',
      'CSP46',
      'CSP47',
      'CSP48',
      'CSP49',
      'CSP50',
      'CSP55',
      'CSP56',
      'CSP57',
      'CSP60',
      'CSP65',
      'CSP66'
      ]
  return h2o.H2OFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], column_names = columns)


@app.route('/predict', methods=['GET', 'POST'])
def predict3():

    try:

      json_input = request.json

      ID = json_input["ID"]
      LicAge = json_input["LicAge"]
      Gender = map_for_dict_Gender(json_input["Gender"])
      MariStat = map_for_dict_MariStat(json_input["MariStat"])
      DrivAge = json_input["DrivAge"]
      HasKmLimit = json_input["HasKmLimit"]
      BonusMalus = json_input["BonusMalus"]
      OutUseNb = json_input["OutUseNb"]
      RiskArea = json_input["RiskArea"]
      VehUsg_Private = f_VehUsage_Private(json_input["VehUsage"])
      VehUsg_Private_trip_to_office = f_VehUsage_Private_trip_to_office(json_input["VehUsage"])
      VehUsg_Professional = f_VehUsage_Professional(json_input["VehUsage"])
      VehUsg_Professional_run = f_VehUsage_Professional_run(json_input["VehUsage"])
      CSP1 = 0
      CSP2 = 0
      CSP3 = 0
      CSP6 = 0
      CSP7 = 0
      CSP20 = 0
      CSP21 = 0
      CSP22 = 0
      CSP26 = 0
      CSP37 = 0
      CSP40 = 0
      CSP42 = 0
      CSP46 = 0
      CSP47 = 0
      CSP48 = 0
      CSP49 = 0
      CSP50 = 0
      CSP55 = 0
      CSP56 = 0
      CSP57 = 0
      CSP60 = 0
      CSP65 = 0
      CSP66 = 0

      hf = return_NewH2o_Frame()

      hf[0, 'LicAge'] = LicAge
      hf[0, 'Gender'] = Gender
      hf[0, 'MariStat'] = MariStat
      hf[0, 'DrivAge'] = DrivAge
      hf[0, 'HasKmLimit'] = HasKmLimit
      hf[0, 'BonusMalus'] = BonusMalus
      hf[0, 'OutUseNb'] = OutUseNb
      hf[0, 'RiskArea'] = RiskArea
      hf[0, 'VehUsg_Private'] = VehUsg_Private
      hf[0, 'VehUsg_Private+trip to office'] = VehUsg_Private_trip_to_office
      hf[0, 'VehUsg_Professional'] = VehUsg_Professional
      hf[0, 'VehUsg_Professional run'] = VehUsg_Professional_run
      hf[0, 'CSP1'] = CSP1
      hf[0, 'CSP2'] = CSP2
      hf[0, 'CSP3'] = CSP3
      hf[0, 'CSP6'] = CSP6
      hf[0, 'CSP7'] = CSP7
      hf[0, 'CSP20'] = CSP20
      hf[0, 'CSP21'] = CSP21
      hf[0, 'CSP22'] = CSP22
      hf[0, 'CSP26'] = CSP26
      hf[0, 'CSP37'] = CSP37
      hf[0, 'CSP40'] = CSP40
      hf[0, 'CSP42'] = CSP42
      hf[0, 'CSP46'] = CSP46
      hf[0, 'CSP47'] = CSP47
      hf[0, 'CSP48'] = CSP48
      hf[0, 'CSP49'] = CSP49
      hf[0, 'CSP50'] = CSP50
      hf[0, 'CSP55'] = CSP55
      hf[0, 'CSP56'] = CSP56
      hf[0, 'CSP57'] = CSP57
      hf[0, 'CSP60'] = CSP60
      hf[0, 'CSP65'] = CSP65
      hf[0, 'CSP66'] = CSP66

      prediction_Poisson = model_glm_poisson.predict(hf)
      value_Poisson  = prediction_Poisson.as_data_frame()['predict'][0]
      prediction_Gamma = model_glm_gamma.predict(hf)
      value_Gamma  = prediction_Gamma.as_data_frame()['predict'][0]
      value_BurningCost = value_Poisson * value_Gamma

      return jsonify({'ID':ID, 'value_Poisson':value_Poisson, 'value_Gamma':value_Gamma, 'value_BurningCost':value_BurningCost}) 
    
    except:
      
      return "Error"



if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://724d0911.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'CSP4': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'CSP5': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'DrivAgeSq': substituting in a column of NaN
  warnings.warn(w)


glm prediction progress: |████████████████████████████████████████████████| 100%


INFO:werkzeug:127.0.0.1 - - [26/Dec/2019 17:46:21] "POST /predict HTTP/1.1" 200 -
